In [31]:
import sys
sys.path.append("../")
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot

In [32]:
from infrastructure.instrument_collection import instrumentCollection as ic

In [33]:
pair = "GBP_JPY"
granularity = "H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")
MA_LIST = [10, 20, 50, 100, 200]

In [34]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [35]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2022-07-27 21:00:00+00:00,165.982,166.104,165.362,165.418
1,2022-07-28 01:00:00+00:00,165.415,165.423,164.534,164.718
2,2022-07-28 05:00:00+00:00,164.716,165.290,164.582,164.906
3,2022-07-28 09:00:00+00:00,164.902,164.902,163.433,163.534
4,2022-07-28 13:00:00+00:00,163.528,163.634,162.864,163.157


In [36]:
for ma in MA_LIST:    
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)

In [37]:
df_plot = df_ma.iloc[:500]

In [38]:
cp = CandlePlot(df_plot)

In [39]:
traces = [ f"MA_{x}" for x in MA_LIST ]

In [40]:
cp.show_plot(line_traces=traces)

In [41]:
MA_S = "MA_50"
MA_L = "MA_200"
BUY = 1
SELL = -1
NONE = 0

In [42]:
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [43]:
df_an.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200
0,2022-09-13 01:00:00+00:00,166.610,166.763,166.424,166.558,164.07226,162.612535
1,2022-09-13 05:00:00+00:00,166.556,166.938,166.367,166.642,164.17380,162.618655
2,2022-09-13 09:00:00+00:00,166.642,168.166,166.134,166.873,164.28096,162.629430
3,2022-09-13 13:00:00+00:00,166.873,166.890,166.120,166.302,164.37782,162.636410
4,2022-09-13 17:00:00+00:00,166.301,166.433,166.010,166.142,164.47282,162.649450


In [44]:
df_an['DELTA'] = df_an.MA_50 - df_an.MA_200
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)

In [45]:
def is_trade(row):
    if row.DELTA >= 0  and row.DELTA_PREV < 0:
        return BUY
    elif row.DELTA < 0  and row.DELTA_PREV >= 0:
        return SELL
    return NONE

In [46]:
df_an['TRADE'] = df_an.apply(is_trade, axis=1)

In [47]:
df_trades = df_an[df_an.TRADE != NONE].copy()

In [48]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
53,2022-09-25 21:00:00+00:00,155.138,155.640,149.333,149.497,162.73530,162.746095,-0.010795,0.256970,-1
122,2022-10-11 09:00:00+00:00,160.909,161.766,160.778,161.505,162.13024,162.123840,0.006400,-0.101630,1
275,2022-11-15 22:00:00+00:00,165.286,165.706,164.708,165.493,166.24186,166.275035,-0.033175,0.000815,-1
400,2022-12-14 18:00:00+00:00,167.771,168.395,167.361,168.376,167.11960,167.093855,0.025745,-0.052325,1
426,2022-12-21 02:00:00+00:00,160.819,161.042,160.488,160.837,166.46322,166.546680,-0.083460,0.022110,-1


In [49]:
cp = CandlePlot(df_an.iloc[220:260])
cp.show_plot(line_traces=[MA_S, MA_L])

In [50]:
ic.LoadInstruments("../data")

In [51]:
ic.instruments_dict[pair]

{'name': 'GBP_JPY', 'ins_type': 'CURRENCY', 'displayName': 'GBP/JPY', 'pipLocation': 0.01, 'tradeUnitsPrecision': 0, 'marginRate': 0.05}

In [52]:
ins_data = ic.instruments_dict[pair]

In [25]:
df_trades.shape

(22, 10)

In [26]:
df_trades['DIFF'] = df_trades.mid_c.diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [27]:
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
df_trades['GAIN'] = df_trades['GAIN'] * df_trades['TRADE'] 

In [28]:
df_trades.GAIN.sum()

np.float64(-2469.0999999999917)

In [29]:
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [30]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=['GAIN_C'])